In [ ]:
import scipy
import numpy as np
from astropy.stats import sigma_clip
from astropy.io import fits
import glob
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval, LinearStretch, ImageNormalize
from photutils import CircularAperture
from photutils import DAOStarFinder
from astropy.io.fits import getheader
from  astropy.io.ascii import SExtractor
from astropy.table import Table 
from astropy.io import ascii
import seaborn as sns
import os
import matplotlib

In [ ]:
def poisson_noise(name,ratio = 2,exp_time = None, ps = None, ron = 3, dk = 7, sv_name = None, path = None, ret = True, save = False):
        """Creating image with poisson noise
       name - path to original image, image is in e/sec
       exp_time - exposure time of the image
       ratio - int number, used to scale exp_time
       ron - read out noise - from hubble handbook
       dk - dark current - from hubble handbook
       sv_name - save name of noisy image
       if save = True - save noisy image
       if ret = True - return image """
    
    img_data = fits.getdata(name) #e/sec
    img_data = np.where(img_data < 0.00000000e+00, 0.00000000e+00,img_data) #to not have minus pixels
    width, height = img_data.shape[0:2]

    img = img_data * exp_time   # e
    
    DN = np.random.normal(0, np.sqrt(dk*exp_time/((60*60)*ratio)), (width, height))    
    RON = np.random.normal(0, ron, (width, height))
    SN = np.random.poisson(np.abs(img/ratio))   # this line is different  
    
    noise_img = (SN + RON + DN)/(exp_time/ratio)
    
    noise_img = np.where(img_data == 0.00000000e+00, 0.00000000e+00, noise_img)
    if save:
        fits.writeto(sv_name , noise_img)
    if ret:    
        return noise_img